Model the correction factor c using a weighted linear regression model

In [ ]:
# use kernel [conda env:ml]
import os
import csv
import _csv
import psutil
import multiprocessing as mp
from typing import TextIO, Union, List

import numpy as np
import osmnx as ox
import pandas as pd
import geopandas as gpd
import pyproj
from collections import Counter
import time
import itertools

#from aggregator_multi import create_graph
# from .projection_data import projection_data_class
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib as mpl
from matplotlib.colors import ListedColormap

import statsmodels.api as sm #for linear regression

from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance #to get permutation importance of variables

#for train test split
from sklearn.model_selection import train_test_split

#for rmse
from statsmodels.tools.eval_measures import rmse

In [ ]:
%store -r Ngraph
%store -r edges2
%store -r nodes
%store -r traffic

In [ ]:
def plot_result(x, y, x_label = "$\sqrt{\hat{c} * exp}$", y_label = "$\sqrt{obs}$", save_fig = False, save_path = None):

    fig, ax = plt.subplots(figsize = (6,4))
    plt.style.use("bmh")
    # Plots #
        # Plot scatter
    plt.scatter(x, y, s = 1)

    #obtain m (slope) and b(intercept) of linear regression line
    m, b = np.polyfit(x, y, 1)

    #add linear regression line to scatterplot 
    plt.plot(x, m*x+b,color = "red")

    # X #
    ax.set_xlabel(x_label)

    # Y #
    #ax.set_yticks([])
        # Relabel the axis as "Frequency"
    ax.set_ylabel(y_label)

    # Overall #
    #ax.set_title("Relationship between observed and expected traffic counts")
    # Remove ticks and spines
    ax.tick_params(left = False, bottom = False)
    for ax, spine in ax.spines.items():
        spine.set_visible(False)

   # add rmse, r^2 and correlation coefficient 
    #first build linear regression model
    X = x
    X = sm.add_constant(X) # adding a constant
    model_plot = sm.OLS(y, X).fit()

    #get pearsons correlation coefficient
    corr_matrix = np.corrcoef(x,y)

    plt.annotate(text = ("$R^2$ = {:.3f}".format(model_plot.rsquared) + "\ncorr = {:.3f}".format(corr_matrix[1, 0])), xy = (.8, .85), xycoords = 'axes fraction')
    plt.grid(False)
    plt.rcParams['font.family'] = ['serif']
    
    if save_fig == True:
        fig.savefig(save_path, bbox_inches = "tight", pad_inches = 0, dpi=10, format = "pdf")
        
    plt.show()

In [ ]:
def result_scatter(c_cb, Xtest = X_test, fig_save = False, path_save = None):
    Xresult = Xtest.copy()
#    c_cb = np.where(c_cb < 0, 0.0001, c_cb)
    Xresult["c_hat"] = c_cb**3
    Xresult["c_hat"] = np.where(Xresult["c_hat"] < 0, 0.0001, Xresult["c_hat"])  # The linear regression may predict a calibration factor that is below 0. Substantially, this can be interpreted as a calibration factor that is very close to 0. We interpret these as just very close to 0
    Xresult["c_exp_car"] = Xresult["exp_car"] * Xresult["c_hat"]

    y = Xresult["obs_car"] ** (1/2) 
    x = Xresult["c_exp_car"] ** (1/2)

    plot_result(x = x, y = y, x_label = "$\sqrt{\hat{c} * exp}$", y_label = "$\sqrt{obs}$", save_fig = fig_save, save_path = path_save)

In [ ]:
def add_features_to_edge(df, edges, features = ["added_feature"], node_from = "node_from", node_to = "node_to", indexerror = np.nan):

    # Code is a simplified and generalized version of Yvonne Gootzen's visualize_intensities function
    edge_idx_0 = [idx[0] for idx, _ in edges.iterrows()]
    edge_idx_1 = [idx[1] for idx, _ in edges.iterrows()]
    
    n_features = len(features)
    features_list = [[] for _ in range(n_features)]
    for nf, nt in zip(edge_idx_0, edge_idx_1):
        for n in range(n_features): #for each feature
    
            try:
                # search in intensities for a have a matching edge
                feat = df.loc[(df[node_from] == nf) & (df[node_to] == nt), features[n]].iloc[0]

                features_list[n].append(feat)

            except IndexError:
                features_list[n].append(indexerror)


    for m in range(n_features):
        edges[features[m]] = features_list[m]
    
    return edges

# Perform validation analysis $V_1$ to get a baseline of the quality of expected counts

This is a linear regression with the observed counts as the outcome and the expected counts as the predictor

In [ ]:
plot_result(x = traffic["exp"], y = traffic["obs"], x_label = "Expected count", y_label = "Observed count", save_fig = False, save_path = None)

We can see that the different spread and distribution of the two data sets results in a non-linear relationship. To alleviate this, we can square-root transform the counts. The $R^2$ of this result will be our baseline.

In [ ]:
traffic["obssq"] = traffic["obs"]**(1/2)
traffic["expsq"] = traffic["exp"]**(1/2)
plot_result(x = traffic["expsq"], y = traffic["obssq"], x_label = "$\sqrt{Expected\ count}$", y_label = "$\sqrt{Observed\ count}$", save_fig = False)

# Compute and inspect the calibration factor $C$
C is the ratio of the observed to the expected count. If c > 1, we have more observed counts than we expect. If c < 1, we have less observed counts than we expect.

In [ ]:
traffic["c"] = traffic["obs"]/traffic["exp"] 

### Histograms

In [ ]:
fig, ax = plt.subplots(figsize = (6,4))
plt.style.use("bmh")
# Plots #
    # Plot histogram
traffic["c"].plot(kind = "hist", density = False, alpha = 0.8, bins = 100) # change density to true, because KDE uses density

# X #
ax.set_xlabel("$c$")

# Y #
#ax.set_yticks([])
    # Relabel the axis as "Frequency"
ax.set_ylabel("Frequency")

# Overall #
ax.set_title("Distribution of calibration factor C (ratio of observed to expected count)")
# Remove ticks and spines
ax.tick_params(left = False, bottom = False)
for ax, spine in ax.spines.items():
    spine.set_visible(False)

plt.grid(False)
plt.rcParams['font.family'] = ['serif']
#fig.savefig("figures/c_hist.pdf", bbox_inches = "tight", pad_inches = 0, dpi=10, format = "pdf")
plt.show()

Right skewed with very long tail, which makes it hard to inspect values below 1. To get a clearer insight, we can perform a cube root transformation on c.

In [ ]:
traffic["c_cb"] = traffic["c"]**(1/3)

fig, ax = plt.subplots(figsize = (6,4))
plt.style.use("bmh")
# Plots #
    # Plot histogram
traffic["c_cb"].plot(kind = "hist", density = False, alpha = 0.8, bins = 100) # change density to true, because KDE uses density

# X #
ax.set_xlabel("$\sqrt[3]{c}$")

# Y #
#ax.set_yticks([])
    # Relabel the axis as "Frequency"
ax.set_ylabel("Frequency")

# Overall #
ax.set_title("Distribution of $\sqrt[3]{.}$-transformed calibration factor C (ratio of observed to expected count)")
# Remove ticks and spines
ax.tick_params(left = False, bottom = False)
for ax, spine in ax.spines.items():
    spine.set_visible(False)

plt.grid(False)
plt.rcParams['font.family'] = ['serif']
#fig.savefig("figures/cuberoot_c_hist.pdf", bbox_inches = "tight", pad_inches = 0, dpi=10, format = "pdf")
plt.show()

### Plot on road network

In [ ]:
from matplotlib.colors import ListedColormap
def plot_examples(colormaps, lo = 1, hi = 100):
    """
    Helper function to plot data with associated colormap.
    """
    np.random.seed(19680801)
    data = np.random.randint(low = lo, high = hi, size = (30,30))
    n = len(colormaps)
    fig, axs = plt.subplots(1, n, figsize=(n * 2 + 2, 3),
                            constrained_layout=True, squeeze=False)
    for [ax, cmap] in zip(axs.flat, colormaps):
        psm = ax.pcolormesh(data, cmap=cmap, rasterized=True, vmin=lo, vmax=hi)
        fig.colorbar(psm, ax=ax)
    plt.show()
#modify colormap 

For this plot, It is best if we add another square root transformation to the cube root transformed C. Generally, I would say that we should inspect C with more data before drawing conclusions from such a highly and somewhat arbitrarily transformed variable, but for the sake of the plot, it does not matter
</br> When we have the transformed variable, we can bin it into 100 bins, meaning that the values will be put into equal sized bins from 1-100. This is useful to set the midpoint of the colorscale, which ranges from 0-1. This will also become useful later, when we want to plot c*, i.e., the ratio after calibration. We will put these bin values into the variable "c_plot"

In [ ]:
traffic["c_sq"] = traffic.c_cb**(1/2)

c_binned = pd.cut(traffic.c_sq, 100, labels = False)

#add to DF but add +1 to each value to keep 0 as lowest
traffic["c_plot"] = c_binned +1

In [ ]:
plt.scatter(traffic["c_plot"], traffic["c_cb"], s = 1) 

A c_plot value of 40 represents a calibration factor of 1, as can be seen in the plot above. This means that the colorscale should have 40 as the midpoint. 

In [ ]:
#create colorscales
# Diverging color scale to see both below and above 1 values, set lowest value to grey colour
# colormap for c
seismic1 = cm.get_cmap('seismic', 101)

# set the linspace from 0.175. This sets the midpoint (which shall represent a ratio of 1) around 40 (i.e. 0.4)
seismic_colors1 = seismic1(np.linspace(0.175, 1, 101)) 

darkgrey = np.array([0.5, 0.5, 0.5, 0.15]) #the fourth value sets the transparency of the color. This is necessary, because the grey roads would otherwise cover up the colored roads
seismic_colors1[:1 :] = darkgrey
seismiccmp1 = ListedColormap(seismic_colors1)

#create colorbar without grey starting value 
seismic_colorsng = seismic1(np.linspace(0.175, 1, 101))
seismiccmp_cb = ListedColormap(seismic_colorsng)

plot_examples([seismic1, seismiccmp1])

Now, we add c and c_plot to edges2, and plot the calibration factor on the Dutch road network afterwards.

In [ ]:
edges2 = add_features_to_edge(traffic, edges2, features = ["c", "c_plot"], indexerror = 0)

In [ ]:
graph = ox.graph_from_gdfs(nodes, edges2)
ec = ox.plot.get_edge_colors_by_attr(graph, attr="c_plot", cmap= seismiccmp1)

fig, ax = ox.plot_graph(graph, node_color="w", node_edgecolor="k", node_size=0, edge_color=ec, edge_linewidth=2, show = False) 

# add colorbar
sm = mpl.cm.ScalarMappable(cmap = seismiccmp_cb)
sm.set_array([])
cb = fig.colorbar(cm.ScalarMappable(cmap = seismiccmp_cb), ax = ax, location = 'right', shrink = 0.8, ticks = [traffic["c_plot"].min()*0.01, 40*0.01, traffic["c_plot"].max()*0.01])
cb.ax.set_yticklabels(['0', '1', '70'])
cb.ax.set_ylabel('C', rotation = 0)


#fig.savefig("playgorund/c_map.pdf", format = "pdf", bbox_inches = "tight", pad_inches = 0, dpi = 1000) #bbox_inches and pad_inches ensure that there is no white frame around the plot
plt.show()

# Modeling C

# Get road segment characteristics
Road segment characteristics are stored in edges (or edges2, respectively), but to use them for modeling we need to to some data management.

In [ ]:
#set highway as category to create dummies 
edges2["highway_old"] = edges2["highway"].astype(str).astype("category")
#There are some instances where the highway is a list of different highway types. We will call these "mixed_highwaytypes"
main_highwaytypes = ['primary', 'trunk', 'primary_link',
       'motorway', 'motorway_link', 'trunk_link']
edges2["highway"] = np.where(~edges2["highway_old"].isin(main_highwaytypes), "mixed_highwaytypes", edges2["highway_old"])


edges2["is_bridge"] = np.where(edges2["bridge"].isna(), 0, 1)

# in some instances, maxspeed column also contains lists with multiple maxspeeds and strings
# we will take the highest value in these instances, and store the new variable as maxmaxspeed
out = edges2.maxspeed.explode().groupby(level = 0).max() 
edges2["maxmaxspeed"] = out
edges2["maxmaxspeed"] = edges2["maxmaxspeed"].astype(float)

# in some instances, lanes column also contains lists with multiple lanes and strings
# we will take the highest value in these instances, and store the new variable as maxlanes
edges2["lanes_chr"] = edges2.lanes.astype(str)
out = edges2.lanes.explode().groupby(level = 0).max() 
edges2["maxlanes"] = out
edges2["maxlanes"] = edges2["maxlanes"].astype(float)

## Get regional characteristics

#### Population density (municipality level)
First we need the shapefile with all the municipalities, then we need the population density in 2019 from opendata.cbs.nl. Finally, we can turn traffic dataframe into a geodataframe and use the geopandas function sjoin to find out which sensor is located in which municipality. The population density can be found here: https://opendata.cbs.nl/statline/#/CBS/nl/dataset/70072ned/table?dl=5A35F. I put the data into a csv that has the municipality on one column and the population density in another column. I also included the total population of each municipality in that csv, but this can be discarded. 

In [ ]:
nlmap_mun = gpd.read_file("Data/CBS/Shapefiles/Municipality/bu_2019.shp")
nlmap_mun = nlmap_mun.to_crs("EPSG:4326") #change CRS to fit the longitude/latitude of sensor data

In [ ]:
popdens = pd.read_csv("Data/CBS/Statline/Regionale_kerncijfers_Nederland_03022022_140149.csv", sep = ";")
popdens = popdens.rename(columns = {"Regio's": "GM_NAAM", "Bevolking/Bevolkingssamenstelling op 1 januari/Totale bevolking (aantal)": "total_pop_municipality", "Bevolking/Bevolkingssamenstelling op 1 januari/Bevolkingsdichtheid (aantal inwoners per km²)": "popdens", "Milieu en bodemgebruik/Bodemgebruik/Oppervlakte/Land (km²)": "area"})

In [ ]:
nlmap_mundens = popdens.merge(nlmap_mun, on = "GM_NAAM", how = "outer")
nlmap_mundens = nlmap_mundens[["GM_NAAM", "geometry", "popdens"]] #remove unncessesary columns

In [ ]:
nlmap_mundens_gdf = gpd.GeoDataFrame(
    nlmap_mundens, geometry="geometry")

Link municipality to sensor: Map sensor to popdens using long/lat and polygon

In [ ]:
traffic[["longitude", "latitude"]] = traffic["coordinates"].str.split(",", expand = True)
traffic["longitude"] = traffic["longitude"].str[1:]
traffic["latitude"] = traffic["latitude"].str.rstrip("]")

In [ ]:
traffic_gdf = gpd.GeoDataFrame(
    traffic, geometry=gpd.points_from_xy(traffic.longitude, traffic.latitude))

traffic_gdf.crs = "EPSG:4326"

In [ ]:
traffic2 = nlmap_mundens_gdf.sjoin(traffic_gdf, how = "right", predicate = "contains")
traffic2 = traffic2.drop(columns = ["index_left"])

In [ ]:
# add to edges2
edges2 = add_features_to_edge(traffic2, edges2, features = ["popdens"], indexerror = np.nan)

# Delete these chunks after final run

In [ ]:
# add to edges2
#edges3 = edges2.set_index('u')
#edges3 = edges3.set_index('v', append=True)
#edges3 = add_features_to_edge(traffic2, edges3, features = ["popdens"], indexerror = np.nan)

In [ ]:
# add to edges2
#edges_model2 = edges_model.set_index('u')
#edges_model2 = edges_model2.set_index('v', append=True)
#edges_model2 = add_features_to_edge(traffic2, edges_model2, features = ["popdens"], indexerror = np.nan)

In [ ]:
#edges_model = edges_model2

In [ ]:
#edges2_old = edges2
#edges2 = pd.DataFrame(edges3.reset_index())

#### Nearness to border
To get the points where Dutch roads cross the border, I use a shapefile that Marko Roos provided me with for a side project. I then check for each road segment if it is intersecting a 12km perimeter around those border points, to create the dummy variable "close_to_border"

In [ ]:
bcsnap = gpd.read_file("/data2/dacimob/Voor_Inan/BorderCrossingsNumber_with_BC_SnapToPRRoad_location.shp")

In [ ]:
#set CRS of gdf to EPSG:28992 which is optimal for finding intersections because it is in meters
bcsnap.crs = "EPSG:4326"
bcsnap = bcsnap.to_crs("EPSG:28992") 

In [ ]:
edges2 = pd.DataFrame(edges2.reset_index()) #necessary to turn u and v into dataframe columns that we can easily access
edges2["uv"] = list(zip(edges2.u, edges2.v)) #create u-v combination variable to have an ID of each road segment
edges2_gdf = gpd.GeoDataFrame(edges2, geometry = edges2.geometry)

In [ ]:
#create 12 km perimeter around each border crossing point
bcsnap_buffered = bcsnap.buffer(12000)

un = bcsnap_buffered.unary_union #create one geometry that is the unary union of all buffer zones
intersectants = edges2_gdf["geometry"].intersection(un) #find all intersecting roads to buffer zone
intersecting_roads = edges2_gdf[~intersectants.is_empty] #if road is not intersecting, it will be empty, therefore remove empty ones

In [ ]:
#create dummy variable for roads that are within the buffer zone
edges2["close_to_border"] = np.where(edges2["uv"].isin(intersecting_roads["uv"]), 1, 0)

## Get road network characteristics
Get the most common type, the total length, the total number, the highest speed limit, the lowest speed limit, the highest number of lanes and the lowest number of lanes of all edges that are within a set perimeter around each edge. I wrote a function for this:

In [ ]:
# function to get buffer area features
# takes two geodataframes with same crs as arguments. 
# edges1 is the gdf of which the edges should be buffered, edges2 is the gdf of which the intersections should be extracted. edges1 and edges2 can be the same object
# If you want to add the derived features to another dataframe, you can optionally define edges3 as that other dataframe
# buffer_m is the distance of the buffering in meters
# edges1 can also be an object with edges that were already buffered, in this case, set unbuffered = False and provide edges3
# returns edges1 as df with additional columns for the buffer area features
def get_buffer_features(edges1, edges2, edges3 = None, buffer_m = 1000, unbuffered = True):
    

    #empty lists to store the features
    types = []
    total_length = []
    nr_edges = []
    max_speed = []
    min_speed = []
    max_lanes = []
    min_lanes = []
    
    # buffer edges if edges1 is not already buffered
    if unbuffered == True:

        #buffer edges, meaning "expand" the boundaries of the geometric shape in all directions by buffer_m meters
        buffered_edges = edges1.buffer(buffer_m)
    else: 
        buffered_edges = edges1

    #loop over buffered edges
    for i in range(0, len(buffered_edges)):
        row = buffered_edges[i:(i+1)].unary_union #extract first buffered edge
        intersectants = edges2["geometry"].intersection(row) #find all intersecting edges to buffer zone
        intersecting_edges = edges2[~intersectants.is_empty] #if edge is not intersecting, it will be empty, therefore remove empty ones

        #most common road type of buffer zone
        Type = intersecting_edges["highway"].value_counts().idxmax()
        types.append(Type)

        #total edge length of buffer zone
        Length = intersecting_edges["length"].sum()
        total_length.append(Length)

        #number of edges in area
        Nr = len(intersecting_edges)
        nr_edges.append(Nr)

        #maxspeed
        maspeed = intersecting_edges.maxmaxspeed.max()
        max_speed.append(maspeed)

        #minspeed
        mispeed = intersecting_edges.maxmaxspeed.min()
        min_speed.append(mispeed)

        #maxlanes
        malane = intersecting_edges.maxlanes.max()
        max_lanes.append(malane)

        #minlanes
        milane = intersecting_edges.maxlanes.min()
        min_lanes.append(milane)


    #add column to edges, buffer_str is buffer_m as character string
    buffer_str = str(buffer_m)
    type_column = "commontype" + buffer_str + "m_bufferzone"
    length_column = "total_length" + buffer_str + "m_bufferzone"
    nr_edges_column = "nr_edges" + buffer_str + "m_bufferzone"
    maxspeed_column = "maxspeed" + buffer_str + "m_bufferzone"
    minspeed_column = "minspeed" + buffer_str + "m_bufferzone"
    maxlanes_collumn = "maxlanes" + buffer_str + "m_bufferzone"
    minlanes_column = "minlanes" + buffer_str + "m_bufferzone"

    if edges3 is None: #if edges3 is not defined, add derived features to edges1
        edges3 = edges1
    
    return_edges = edges3
    

    return_edges[type_column] = types
    return_edges[length_column] = total_length
    return_edges[nr_edges_column] = nr_edges
    return_edges[maxspeed_column] = max_speed
    return_edges[minspeed_column] = min_speed
    return_edges[maxlanes_collumn] = max_lanes
    return_edges[minlanes_column] = min_lanes
    
    return return_edges
    

Because there are 68000 edges on the Dutch road network, this is a slow process. To speed it up, we can use multiprocessing. We will also only get these values for roads that also have observed counts, to speed things up. However, later this needs to be applied to the entire set of edges, because we will need these road network characteristics to calibrate all expected counts. 

In [ ]:
edges_network = edges2.loc[~(edges2.c == 0)]

In [ ]:
# create geodataframes
edges_network_gdf = gpd.GeoDataFrame(
    edges_network, geometry=edges_network.geometry)

edges2_gdf = gpd.GeoDataFrame(
    edges2, geometry=edges2.geometry)

In [ ]:
#wrapper function for get_buffer_features using multiprocessing
def get_buffer_features_pool1500(edges1_pool):
    
    buffered_with_features = get_buffer_features(edges1 = edges1_pool, edges2 = edges2_gdf, buffer_m = 1500, unbuffered = True)
    
    return buffered_with_features

def get_buffer_features_pool12000(edges1_pool):
    
    buffered_with_features = get_buffer_features(edges1 = edges1_pool, edges2 = edges2_gdf, buffer_m = 12000, unbuffered = True)
    
    return buffered_with_features

In [ ]:
#proceed with full dataset
NUM_CORES = mp.cpu_count() #the number of processing cores in zweistein
dat_chunks = np.array_split(edges_network_gdf, NUM_CORES)

with mp.Pool(NUM_CORES) as pool:
    
     edges_network12000 = pd.concat(pool.map(get_buffer_features_pool12000, dat_chunks), ignore_index = True)
        
with mp.Pool(NUM_CORES) as pool:
    
     edges_network1500 = pd.concat(pool.map(get_buffer_features_pool1500, dat_chunks), ignore_index = True)

In [ ]:
# remove most columns that are not unique, because .merge() can only take a few key variables
edges_network1500_thin = edges_network1500[['obs', 'uv', 'commontype1500m_bufferzone',
       'total_length1500m_bufferzone', 'nr_edges1500m_bufferzone',
       'maxspeed1500m_bufferzone', 'minspeed1500m_bufferzone',
       'maxlanes1500m_bufferzone', 'minlanes1500m_bufferzone']]

edges_model = edges_network12000.merge(edges_network1500_thin, on = ["uv", "obs"], how = "inner")
edges_model.to_csv("edges_network_full.csv", index = False)

In [ ]:
edges_model.columns

In [ ]:
edges_model

## Add province to edges

In [ ]:
edges_model = add_features_to_edge(traffic2, edges_model, features = ["PV_NAAM"], indexerror = np.nan)

# Model C

Because of the skewed distribution of C, we will use the cube root transformed C for modeling (this is not necessary in the case of random forest, but for comparison, it is best to have the same outcome in both models). The expected counts have an inverse squared relationship with C (and $\sqrt[3]{C}$). We will therefore include exp$^{-1}$ in the linear regression model. 

In [ ]:
edges_model["c_cb"] = edges_model["c"]**(1/3)
plt.scatter(edges_model["exp"], edges_model["c_cb"])
plt.title("Relationship between exp and $\sqrt[3]{c}$")

## Split data into a train and test set
We use 60% of the data for training and 40% for testing

In [ ]:
edges_model = pd.DataFrame(edges_model.reset_index())

In [ ]:
#segment characteristics
modeldat = pd.get_dummies(edges_model['highway'])
modeldat["maxmaxspeed"] = edges_model["maxmaxspeed"]
modeldat["c_cb"] = edges_model["c_cb"]
modeldat["maxlanes"] = edges_model["maxlanes"]
modeldat["is_bridge"] = edges_model["is_bridge"]

#regional characteristics
modeldat["popdens"] = edges_model["popdens"]
modeldat["close_to_border"] = edges_model["close_to_border"]

#road network characteristics
modeldat["total_length12000m_bufferzone"] = edges_model["total_length12000m_bufferzone"]
modeldat["nr_edges12000m_bufferzone"] = edges_model["nr_edges12000m_bufferzone"]
modeldat["maxspeed12000m_bufferzone"] = edges_model["maxspeed12000m_bufferzone"]
modeldat["minspeed12000m_bufferzone"] = edges_model["minspeed12000m_bufferzone"]
modeldat["maxlanes1500m_bufferzone"] = edges_model["maxlanes1500m_bufferzone"]
modeldat["maxlanes12000m_bufferzone"] = edges_model["maxlanes12000m_bufferzone"]
modeldat["minlanes12000m_bufferzone"] = edges_model["minlanes12000m_bufferzone"]

#province
regional_dummies = pd.get_dummies(edges_model['PV_NAAM'])
modeldat = pd.concat([modeldat, regional_dummies], axis = 1)

#interaction variables
modeldat["popdens_motorway"] = modeldat["motorway"] * modeldat["popdens"]
modeldat["maxlanes_maxlanes1500"] = modeldat["maxlanes1500m_bufferzone"] * modeldat["maxlanes"]

modeldat["popdens_Zuid-Holland"] = modeldat["Zuid-Holland"] * modeldat["popdens"]
modeldat["popdens_Noord-Holland"] = modeldat["Noord-Holland"] * modeldat["popdens"]
modeldat["popdens_Utrecht"] = modeldat["Utrecht"] * modeldat["popdens"]

modeldat["maxlanes12000_Zuid-Holland"] = modeldat["Zuid-Holland"] * modeldat["maxlanes12000m_bufferzone"]
modeldat["maxlanes12000_Noord-Holland"] = modeldat["Noord-Holland"] * modeldat["maxlanes12000m_bufferzone"]
modeldat["maxlanes12000_Utrecht"] = modeldat["Utrecht"] * modeldat["maxlanes12000m_bufferzone"]
modeldat["maxlanes12000_Noord-Brabant"] = modeldat["Noord-Brabant"] * modeldat["maxlanes12000m_bufferzone"]

#expected and observed cars
modeldat["exp_car"] = edges_model["exp"]
modeldat["obs_car"] = edges_model["obs"]
modeldat["exp_car_isq"] = 1 / modeldat["exp_car"] #this is necessary for linear regression, will be shown in the next couple of chunks

#add uv to later merge again with edges_network_full
modeldat["uv"] = edges_model["uv"]
modeldat = modeldat.dropna()

y = modeldat["c_cb"]
X = modeldat
X = sm.add_constant(X, has_constant = 'add') # adding a constant for linear regression

#split into train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.6, random_state = 120)

#save copy of testdata to keep car counts and then remove those variables from X_test 
testdata = X_test

In [ ]:
#### Wrapper function to run the linear regression and checks results with diffrent input
#takes the predictors Xtrain and the outcome ytrain to model the linear regression
#drop is a list of columns that shall be excluded from the predictors
#predicts y_hat using predictors Xtest (from a test set) and computes root mean squared error using ytrain
#optionally, you can perform a weighted linear regression by adding a vector of weights in W (by default, this is a vector of 1's, which is equivalent to unweighted regression)
#returns a list with results in the following order: model, rmse of test set, predicted y of test set, rmse of train set, predicted y of train set. 
def my_wls(Xtrain = X_train, Xtest = X_test, ytrain = y_train, ytest = y_test, drop = [], W = [1]* len(X_train)):
    Xtrain1 = Xtrain.drop(columns = drop)
    Xtest1 = Xtest.drop(columns = drop)
    wlsmodel = sm.WLS(ytrain.astype(float), Xtrain1.astype(float), weights = W).fit()

    print_model = wlsmodel.summary()

    #rmse test
    yhat_test = wlsmodel.predict(Xtest1)
    rmsewls_test = rmse(yhat_test, ytest)
    
    #rmse train
    yhat_train = wlsmodel.predict(Xtrain1)
    rmsewls_train = rmse(yhat_train, ytrain)
    
    return([wlsmodel, rmsewls_test, yhat_test, rmsewls_train, yhat_train])

In [ ]:
drop_columns = ['mixed_highwaytypes', 'motorway', 'motorway_link', 'primary',
       'primary_link', 'trunk', 'trunk_link', "c_cb", 'maxlanes',
       'popdens', 'close_to_border', 'nr_edges12000m_bufferzone',
       'maxspeed12000m_bufferzone', 'minspeed12000m_bufferzone',
       'maxlanes1500m_bufferzone', 'maxlanes12000m_bufferzone', 'Drenthe',
       'Flevoland', 'Friesland', 'Gelderland', 'Groningen', 'Limburg',
       'Noord-Brabant', 'Noord-Holland', 'Overijssel', 'Utrecht', 'Zeeland',
       'Zuid-Holland', 'popdens_motorway', 'maxlanes_maxlanes1500',
       'popdens_Zuid-Holland', 'popdens_Noord-Holland', 'popdens_Utrecht',
       'maxlanes12000_Zuid-Holland', 'maxlanes12000_Noord-Holland',
       'maxlanes12000_Utrecht', 'maxlanes12000_Noord-Brabant', 
       'obs_car', 'is_bridge', 'maxmaxspeed', 'total_length12000m_bufferzone', 'minlanes12000m_bufferzone', 'uv']
wls0 = my_wls(X_train, X_test, y_train, y_test, drop = drop_columns)
print("RMSE on test set = ", wls0[1])
print("RMSE on train set = ", wls0[3])

In [ ]:
print(wls0[0].summary())

In [ ]:
result_scatter(wls0[2])

#### add highway type

In [ ]:
drop_columns = ["primary", "c_cb", 'maxlanes',
       'popdens', 'close_to_border', 'nr_edges12000m_bufferzone',
       'maxspeed12000m_bufferzone', 'minspeed12000m_bufferzone',
       'maxlanes1500m_bufferzone', 'maxlanes12000m_bufferzone', 'Drenthe',
       'Flevoland', 'Friesland', 'Gelderland', 'Groningen', 'Limburg',
       'Noord-Brabant', 'Noord-Holland', 'Overijssel', 'Utrecht', 'Zeeland',
       'Zuid-Holland', 'popdens_motorway', 'maxlanes_maxlanes1500',
       'popdens_Zuid-Holland', 'popdens_Noord-Holland', 'popdens_Utrecht',
       'maxlanes12000_Zuid-Holland', 'maxlanes12000_Noord-Holland',
       'maxlanes12000_Utrecht', 'maxlanes12000_Noord-Brabant','minlanes12000m_bufferzone',
       'obs_car', 'is_bridge', 'maxmaxspeed', 'total_length12000m_bufferzone', 'uv']
wls1 = my_wls(X_train, X_test, y_train, y_test, drop = drop_columns, W = W)
print("RMSE on test set = ", wls1[1])
print("RMSE on train set = ", wls1[3])

In [ ]:
result_scatter(wls1[2])

#### model adding maxlanes


In [ ]:
drop_columns = ["primary", "c_cb", 
       'popdens', 'close_to_border', 'nr_edges12000m_bufferzone',
       'maxspeed12000m_bufferzone', 'minspeed12000m_bufferzone',
       'maxlanes1500m_bufferzone', 'maxlanes12000m_bufferzone', 'Drenthe',
       'Flevoland', 'Friesland', 'Gelderland', 'Groningen', 'Limburg',
       'Noord-Brabant', 'Noord-Holland', 'Overijssel', 'Utrecht', 'Zeeland',
       'Zuid-Holland', 'popdens_motorway', 'maxlanes_maxlanes1500',
       'popdens_Zuid-Holland', 'popdens_Noord-Holland', 'popdens_Utrecht',
       'maxlanes12000_Zuid-Holland', 'maxlanes12000_Noord-Holland',
       'maxlanes12000_Utrecht', 'maxlanes12000_Noord-Brabant', 
       'obs_car', 'is_bridge', 'maxmaxspeed', 'total_length12000m_bufferzone', 'uv', 'minlanes12000m_bufferzone']

wls3 = my_wls(X_train, X_test, y_train, y_test, drop = drop_columns, W = W)
print("RMSE on test set = ", wls3[1])
print("RMSE on train set = ", wls3[3])

In [ ]:
result_scatter(wls3[2])

#### adding popdens

In [ ]:
drop_columns = ["primary", "c_cb", 
       'close_to_border', 'nr_edges12000m_bufferzone',
       'maxspeed12000m_bufferzone', 'minspeed12000m_bufferzone',
       'maxlanes1500m_bufferzone', 'maxlanes12000m_bufferzone', 'Drenthe',
       'Flevoland', 'Friesland', 'Gelderland', 'Groningen', 'Limburg',
       'Noord-Brabant', 'Noord-Holland', 'Overijssel', 'Utrecht', 'Zeeland',
       'Zuid-Holland', 'popdens_motorway', 'maxlanes_maxlanes1500',
       'popdens_Zuid-Holland', 'popdens_Noord-Holland', 'popdens_Utrecht',
       'maxlanes12000_Zuid-Holland', 'maxlanes12000_Noord-Holland',
       'maxlanes12000_Utrecht', 'maxlanes12000_Noord-Brabant',
       'obs_car', 'is_bridge', 'maxmaxspeed', 'total_length12000m_bufferzone', 'uv', 'minlanes12000m_bufferzone']

wls4 = my_wls(X_train, X_test, y_train, y_test, drop = drop_columns, W = W)
print("RMSE on test set = ", wls4[1])
print("RMSE on train set = ", wls4[3])

In [ ]:
result_scatter(wls4[2])

#### adding interaction between popdens and motorway and adding close-to-border

In [ ]:
drop_columns = ["primary", "c_cb", 
       'nr_edges12000m_bufferzone',
       'maxspeed12000m_bufferzone', 'minspeed12000m_bufferzone',
       'maxlanes1500m_bufferzone', 'maxlanes12000m_bufferzone', 'Drenthe',
       'Flevoland', 'Friesland', 'Gelderland', 'Groningen', 'Limburg',
       'Noord-Brabant', 'Noord-Holland', 'Overijssel', 'Utrecht', 'Zeeland',
       'Zuid-Holland', 'maxlanes_maxlanes1500',
       'popdens_Zuid-Holland', 'popdens_Noord-Holland', 'popdens_Utrecht',
       'maxlanes12000_Zuid-Holland', 'maxlanes12000_Noord-Holland',
       'maxlanes12000_Utrecht', 'maxlanes12000_Noord-Brabant', 
       'obs_car', 'is_bridge', 'maxmaxspeed', 'total_length12000m_bufferzone', 'uv', 'minlanes12000m_bufferzone']

wls5 = my_wls(X_train, X_test, y_train, y_test, drop = drop_columns, W = W)
print("RMSE on test set = ", wls5[1])
print("RMSE on train set = ", wls5[3])

In [ ]:
result_scatter(wls5[2])

#### adding characteristics of surrounding rode network 

In [ ]:
drop_columns = ["primary", "c_cb", 
       'Drenthe',
       'Flevoland', 'Friesland', 'Gelderland', 'Groningen', 'Limburg',
       'Noord-Brabant', 'Noord-Holland', 'Overijssel', 'Utrecht', 'Zeeland',
       'Zuid-Holland', 'maxlanes_maxlanes1500',
       'popdens_Zuid-Holland', 'popdens_Noord-Holland', 'popdens_Utrecht',
       'maxlanes12000_Zuid-Holland', 'maxlanes12000_Noord-Holland',
       'maxlanes12000_Utrecht', 'maxlanes12000_Noord-Brabant', 
       'obs_car', 'is_bridge', 'maxmaxspeed', 'total_length12000m_bufferzone', 'uv']

wls6 = my_wls(X_train, X_test, y_train, y_test, drop = drop_columns, W = W)
print("RMSE on test set = ", wls6[1])
print("RMSE on train set = ", wls6[3])

In [ ]:
result_scatter(wls6[2])

In [ ]:
print(wls6[0].summary())

#### adding interaction between maxlanes and maxlanes1500

In [ ]:
drop_columns = ["primary", "c_cb", 
       'Drenthe',
       'Flevoland', 'Friesland', 'Gelderland', 'Groningen', 'Limburg',
       'Noord-Brabant', 'Noord-Holland', 'Overijssel', 'Utrecht', 'Zeeland',
       'Zuid-Holland', 
       'popdens_Zuid-Holland', 'popdens_Noord-Holland', 'popdens_Utrecht',
       'maxlanes12000_Zuid-Holland', 'maxlanes12000_Noord-Holland',
       'maxlanes12000_Utrecht', 'maxlanes12000_Noord-Brabant', 
       'obs_car', 'is_bridge', 'maxmaxspeed', 'total_length12000m_bufferzone', 'uv']

wls7 = my_wls(X_train, X_test, y_train, y_test, drop = drop_columns, W = W)
print("RMSE on test set = ", wls7[1])
print("RMSE on train set = ", wls7[3])

In [ ]:
result_scatter(wls7[2])

#### adding regions

In [ ]:
drop_columns = ["primary", "c_cb", 
       'Drenthe',
       'popdens_Zuid-Holland', 'popdens_Noord-Holland', 'popdens_Utrecht',
       'maxlanes12000_Zuid-Holland', 'maxlanes12000_Noord-Holland',
       'maxlanes12000_Utrecht', 'maxlanes12000_Noord-Brabant',
       'obs_car', 'is_bridge', 'maxmaxspeed', 'total_length12000m_bufferzone', 'uv']

wls8 = my_wls(X_train, X_test, y_train, y_test, drop = drop_columns, W = W)
print("RMSE on test set = ", wls8[1])
print("RMSE on train set = ", wls8[3])

In [ ]:
result_scatter(wls8[2])

#### final model: adding interaction with regions

In [ ]:
drop_columns = ["primary", "c_cb", 
       'Drenthe', 
       'obs_car', 'is_bridge', 'maxmaxspeed', 'total_length12000m_bufferzone', 'uv']

wls9 = my_wls(X_train, X_test, y_train, y_test, drop = drop_columns, W = W)
print(wls9[0].summary())
print("RMSE on test set = ", wls9[1])
print("RMSE on train set = ", wls9[3])

In [ ]:
#print results as csv table to built appendix B 
#print(wls9[0].summary().as_csv())

In [ ]:
result_scatter(wls9[2])

#### Check multicolinearity
most VIFs are way below 5, except for some of the dummies of regions (Noord-Holland, Noord-Brabant, Utrecht, Zuid-Holland). However, because they are dummies, they will naturally be correlated with other dummies.

In [ ]:
# check multicolinearity
from statsmodels.stats.outliers_influence import variance_inflation_factor

X_variables = X_train.drop(columns = [#'mixed_highwaytypes', 'motorway_link', 'primary_link', 'trunk', 'trunk_link', 
                                      "primary", "c_cb", 
       'Drenthe', 
       'obs_car',  'popdens_motorway', 'maxlanes_maxlanes1500',
       'popdens_Zuid-Holland', 'popdens_Noord-Holland', 'popdens_Utrecht',
       'maxlanes12000_Zuid-Holland', 'maxlanes12000_Noord-Holland',
       'maxlanes12000_Utrecht', 'maxlanes12000_Noord-Brabant', 'total_length12000m_bufferzone', 'uv' ])

vif_data = pd.DataFrame()
vif_data["feature"] = X_variables.columns
vif_data["VIF"] = [variance_inflation_factor(X_variables.values, i) for i in range(len(X_variables.columns))]
print(vif_data) #most VIFs are way below 5, except for some of the dummies of regions (Noord-Holland, Noord-Brabant, Utrecht, Zuid-Holland) and nr_edges_12000

#### check error distribution

In [ ]:
res = wls9[0].resid # residuals
fig = sm.qqplot(res)
plt.title("QQplot of residuals")
plt.show() #right skewedness

#### check for linearity and homoskedasticity

In [ ]:
res = wls9[0].resid # residuals
fitted = wls9[4] #fitted values
plt.title("Fitted vs. residuals")
import seaborn as sns
sns.residplot(x = fitted, y = res, lowess = True, line_kws={'color': 'red', 'lw': 1, 'alpha': 1})

In [ ]:
norm_res_abs_sqrt = np.sqrt(np.abs(res)) #square root standardized residuals

plt.figure(figsize = (7,7))
plt.title("fitted vs standardized residuals")
sns.regplot(fitted, norm_res_abs_sqrt, scatter = True, lowess = True,  line_kws={'color': 'red', 'lw': 1, 'alpha': 1})

#### test for heteroskedasticity

In [ ]:
from statsmodels.stats.diagnostic import het_white
from statsmodels.stats.diagnostic import het_breuschpagan
white_test = het_white(res, wls9[0].model.exog)

X_variables = X_train.drop(columns = ["primary", "c_cb", 
       'Drenthe', 'exp_car',
       'obs_car' , 'uv'])
bp_test = het_breuschpagan(res, X_variables)

In [ ]:
label_test = ["LM statistic", "LM test p value", "F statistic", "F test p value"]
print(dict(zip(label_test, bp_test)))
print(dict(zip(label_test, white_test)))

#### same with random forest

In [ ]:
def my_rf(Xtrain = X_train, Xtest = X_test, ytrain = y_train, ytest = y_test, seed = 4, n_trees = 500, drop = drop_columns, W = W, min_split = 2, min_leaf = 1):
    X_train_rf = Xtrain.drop(columns = drop)
    X_test_rf = Xtest.drop(columns = drop)

    # Instantiate model with 500 decision trees
    ranfor = RandomForestRegressor(n_estimators = n_trees, random_state = seed, min_samples_split=min_split, min_samples_leaf=min_leaf)
    
    # Train the model on training data
    ranfor1 = ranfor.fit(X_train_rf, ytrain)
    
    # Use the forest's predict method on the test data
    predictions_train = ranfor1.predict(X_train_rf)
    # Calculate the absolute errors
    rmse_rf_train = rmse(predictions_train, ytrain)

    # Use the forest's predict method on the test data
    predictions_test = ranfor1.predict(X_test_rf)
    # Calculate the root mean squared error
    rmse_rf_test = rmse(predictions_test, ytest)
    
    importance_plot_gini = plt.barh(X_train_rf.columns, ranfor1.feature_importances_)
    importance_table_gini = pd.DataFrame({"variable": X_train_rf.columns, "importance": ranfor1.feature_importances_}).sort_values(by = ["importance"], ascending = False)
    
    perm_importance = permutation_importance(ranfor1, X_test_rf, y_test, n_repeats=10, random_state=4, n_jobs=2)
    importance_table_permutation = pd.DataFrame({"variable": X_test_rf.columns, "importance": perm_importance.importances_mean}).sort_values(by = ["importance"], ascending = False)
    importance_table_permutation["normalized_importance"] = ( importance_table_permutation["importance"] - importance_table_permutation["importance"].min() ) / ( importance_table_permutation["importance"].max() - importance_table_permutation["importance"].min() )
    
    return([ranfor1, rmse_rf_test, predictions_test, importance_plot_gini, importance_table_gini, importance_table_permutation, rmse_rf_train])


In [ ]:
X_train.columns

In [ ]:
#### exp counts only
drop_columns = ['const', 'mixed_highwaytypes', 'motorway', 'motorway_link', 'primary',
       'primary_link', 'trunk', 'trunk_link', 'c_cb', 'maxlanes',
       'popdens', 'close_to_border', 'nr_edges12000m_bufferzone',
       'maxspeed12000m_bufferzone', 'minspeed12000m_bufferzone',
       'maxlanes1500m_bufferzone', 'maxlanes12000m_bufferzone', 'Drenthe',
       'Flevoland', 'Friesland', 'Gelderland', 'Groningen', 'Limburg',
       'Noord-Brabant', 'Noord-Holland', 'Overijssel', 'Utrecht', 'Zeeland',
       'Zuid-Holland', 'popdens_motorway', 'maxlanes_maxlanes1500',
       'popdens_Zuid-Holland', 'popdens_Noord-Holland', 'popdens_Utrecht',
       'maxlanes12000_Zuid-Holland', 'maxlanes12000_Noord-Holland',
       'maxlanes12000_Utrecht', 'maxlanes12000_Noord-Brabant', 'exp_car_isq',
       'obs_car', 'uv', 'minlanes12000m_bufferzone', 'total_length12000m_bufferzone', 'is_bridge', 'maxmaxspeed']
rf0 = my_rf(X_train, X_test, y_train, y_test, drop = drop_columns, min_split = 15, min_leaf = 8)
rf0[6] #rmse on train set

In [ ]:
rf0[1] #rmse on test set

In [ ]:
rf0[5]

In [ ]:
result_scatter(rf0[2])

#### highway type

In [ ]:
drop_columns = ['const', 'c_cb', 'maxlanes',
       'popdens', 'close_to_border', 'nr_edges12000m_bufferzone',
       'maxspeed12000m_bufferzone', 'minspeed12000m_bufferzone',
       'maxlanes1500m_bufferzone', 'maxlanes12000m_bufferzone', 'Drenthe',
       'Flevoland', 'Friesland', 'Gelderland', 'Groningen', 'Limburg',
       'Noord-Brabant', 'Noord-Holland', 'Overijssel', 'Utrecht', 'Zeeland',
       'Zuid-Holland', 'popdens_motorway', 'maxlanes_maxlanes1500',
       'popdens_Zuid-Holland', 'popdens_Noord-Holland', 'popdens_Utrecht',
       'maxlanes12000_Zuid-Holland', 'maxlanes12000_Noord-Holland',
       'maxlanes12000_Utrecht', 'maxlanes12000_Noord-Brabant', 'uv',
       'exp_car_isq','obs_car', 'uv', 'minlanes12000m_bufferzone', 'total_length12000m_bufferzone', 'is_bridge', 'maxmaxspeed']
rf1 = my_rf(X_train, X_test, y_train, y_test, drop = drop_columns, min_split = 15, min_leaf = 8)
rf1[6] #rmse on train set
#plt.scatter(test1[2], y_test)

In [ ]:
rf1[1] #rmse on test set

In [ ]:
rf1[5]

In [ ]:
X_result = X_test.copy()
X_result["ratio_hat"] = rf1[2]**3
X_result["c_exp_car"] = X_result["exp_car"] * X_result["ratio_hat"] #weight is expected car

y = X_result["obs_car"] ** (1/2) 
x = X_result["c_exp_car"] ** (1/2)

plot_result(x = x, y = y, x_label = "$\sqrt{\hat{c} * exp}$", y_label = "$\sqrt{obs}$", save_fig = False, save_path = None)

#### add maxplanes

In [ ]:
drop_columns = ['const', 'c_cb',
       'popdens', 'close_to_border', 'nr_edges12000m_bufferzone',
       'maxspeed12000m_bufferzone', 'minspeed12000m_bufferzone',
       'maxlanes1500m_bufferzone', 'maxlanes12000m_bufferzone', 'Drenthe',
       'Flevoland', 'Friesland', 'Gelderland', 'Groningen', 'Limburg',
       'Noord-Brabant', 'Noord-Holland', 'Overijssel', 'Utrecht', 'Zeeland',
       'Zuid-Holland', 'popdens_motorway', 'maxlanes_maxlanes1500',
       'popdens_Zuid-Holland', 'popdens_Noord-Holland', 'popdens_Utrecht',
       'maxlanes12000_Zuid-Holland', 'maxlanes12000_Noord-Holland',
       'maxlanes12000_Utrecht', 'maxlanes12000_Noord-Brabant', 'uv',
       'exp_car_isq','obs_car', 'uv', 'minlanes12000m_bufferzone', 'total_length12000m_bufferzone', 'is_bridge', 'maxmaxspeed']
rf2 = my_rf(X_train, X_test, y_train, y_test, drop = drop_columns, min_split = 15, min_leaf = 8)
rf2[6]

In [ ]:
rf2[1]

In [ ]:
rf2[5].head()

In [ ]:
X_result = X_test.copy()
X_result["ratio_hat"] = rf2[2]**3
X_result["c_exp_car"] = X_result["exp_car"] * X_result["ratio_hat"] #weight is expected car

y = X_result["obs_car"] ** (1/2) 
x = X_result["c_exp_car"] ** (1/2)

plot_result(x = x, y = y, x_label = "$\sqrt{\hat{c} * exp}$", y_label = "$\sqrt{obs}$", save_fig = False, save_path = None)

#### add popdens

In [ ]:
drop_columns = ['const', 'c_cb',
    'close_to_border', 'nr_edges12000m_bufferzone',
       'maxspeed12000m_bufferzone', 'minspeed12000m_bufferzone',
       'maxlanes1500m_bufferzone', 'maxlanes12000m_bufferzone', 'Drenthe',
       'Flevoland', 'Friesland', 'Gelderland', 'Groningen', 'Limburg',
       'Noord-Brabant', 'Noord-Holland', 'Overijssel', 'Utrecht', 'Zeeland',
       'Zuid-Holland', 'popdens_motorway', 'maxlanes_maxlanes1500',
       'popdens_Zuid-Holland', 'popdens_Noord-Holland', 'popdens_Utrecht',
       'maxlanes12000_Zuid-Holland', 'maxlanes12000_Noord-Holland',
       'maxlanes12000_Utrecht', 'maxlanes12000_Noord-Brabant', 'uv',
       'exp_car_isq','obs_car', 'uv', 'minlanes12000m_bufferzone', 'total_length12000m_bufferzone', 'maxmaxspeed', 'is_bridge']
rf3 = my_rf(X_train, X_test, y_train, y_test, drop = drop_columns, min_split = 15, min_leaf = 8)
rf3[6]

In [ ]:
rf3[1]

In [ ]:
rf3[5].head()

In [ ]:
X_result = X_test.copy()
X_result["ratio_hat"] = rf3[2]**3
X_result["c_exp_car"] = X_result["exp_car"] * X_result["ratio_hat"] #weight is expected car

y = X_result["obs_car"] ** (1/2) 
x = X_result["c_exp_car"] ** (1/2)

plot_result(x = x, y = y, x_label = "$\sqrt{\hat{c} * exp}$", y_label = "$\sqrt{obs}$", save_fig = False, save_path = None)

#### add border closeness

In [ ]:
drop_columns = ['const', 'c_cb',
    'nr_edges12000m_bufferzone',
       'maxspeed12000m_bufferzone', 'minspeed12000m_bufferzone',
       'maxlanes1500m_bufferzone', 'maxlanes12000m_bufferzone', 'Drenthe',
       'Flevoland', 'Friesland', 'Gelderland', 'Groningen', 'Limburg',
       'Noord-Brabant', 'Noord-Holland', 'Overijssel', 'Utrecht', 'Zeeland',
       'Zuid-Holland', 'popdens_motorway', 'maxlanes_maxlanes1500',
       'popdens_Zuid-Holland', 'popdens_Noord-Holland', 'popdens_Utrecht',
       'maxlanes12000_Zuid-Holland', 'maxlanes12000_Noord-Holland',
       'maxlanes12000_Utrecht', 'maxlanes12000_Noord-Brabant', 'uv',
       'exp_car_isq','obs_car', 'uv', 'minlanes12000m_bufferzone', 'total_length12000m_bufferzone', 'maxmaxspeed', 'is_bridge']
rf4 = my_rf(X_train, X_test, y_train, y_test, drop = drop_columns, min_split = 15, min_leaf = 8)
rf4[6]

In [ ]:
rf4[1]

In [ ]:
rf4[5].head()

In [ ]:
X_result = X_test.copy()
X_result["ratio_hat"] = rf4[2]**3
X_result["c_exp_car"] = X_result["exp_car"] * X_result["ratio_hat"] #weight is expected car

y = X_result["obs_car"] ** (1/2) 
x = X_result["c_exp_car"] ** (1/2)

plot_result(x = x, y = y, x_label = "$\sqrt{\hat{c} * exp}$", y_label = "$\sqrt{obs}$", save_fig = False, save_path = None)

#### add road net characteristics

In [ ]:
drop_columns = ['const', 'c_cb',
     'Drenthe',
       'Flevoland', 'Friesland', 'Gelderland', 'Groningen', 'Limburg',
       'Noord-Brabant', 'Noord-Holland', 'Overijssel', 'Utrecht', 'Zeeland',
       'Zuid-Holland', 'popdens_motorway', 'maxlanes_maxlanes1500',
       'popdens_Zuid-Holland', 'popdens_Noord-Holland', 'popdens_Utrecht',
       'maxlanes12000_Zuid-Holland', 'maxlanes12000_Noord-Holland',
       'maxlanes12000_Utrecht', 'maxlanes12000_Noord-Brabant', 'uv',
       'exp_car_isq','obs_car', 'uv', 'minlanes12000m_bufferzone', 'total_length12000m_bufferzone', 'maxmaxspeed', 'is_bridge']
rf5 = my_rf(X_train, X_test, y_train, y_test, drop = drop_columns, min_split = 15, min_leaf = 8)
rf5[6]

In [ ]:
rf5[1]

In [ ]:
rf5[5].head()

In [ ]:
X_result = X_test.copy()
X_result["ratio_hat"] = rf5[2]**3
X_result["c_exp_car"] = X_result["exp_car"] * X_result["ratio_hat"] #weight is expected car

y = X_result["obs_car"] ** (1/2) 
x = X_result["c_exp_car"] ** (1/2)

plot_result(x = x, y = y, x_label = "$\sqrt{\hat{c} * exp}$", y_label = "$\sqrt{obs}$", save_fig = False, save_path = None)

#### add regions

In [ ]:
drop_columns = ['const', 'c_cb',
      'popdens_motorway', 'maxlanes_maxlanes1500',
       'popdens_Zuid-Holland', 'popdens_Noord-Holland', 'popdens_Utrecht',
       'maxlanes12000_Zuid-Holland', 'maxlanes12000_Noord-Holland',
       'maxlanes12000_Utrecht', 'maxlanes12000_Noord-Brabant', 
       'exp_car_isq','obs_car', 'uv', 'minlanes12000m_bufferzone', 'total_length12000m_bufferzone', 'maxmaxspeed', 'is_bridge', "uv"]
rf6 = my_rf(X_train, X_test, y_train, y_test, drop = drop_columns, min_split = 15, min_leaf = 8)
rf6[6]

In [ ]:
rf6[1]

In [ ]:
#Save the importance as an excel to built the table in Appendix B
#rf6[5].to_excel("rf6_importance.xlsx")

In [ ]:
X_result = X_test.copy()
X_result["ratio_hat"] = rf5[2]**3
X_result["c_exp_car"] = X_result["exp_car"] * X_result["ratio_hat"] #weight is expected car

y = X_result["obs_car"] ** (1/2) 
x = X_result["c_exp_car"] ** (1/2)

plot_result(x = x, y = y, x_label = "$\sqrt{\hat{c} * exp}$", y_label = "$\sqrt{obs}$", save_fig = True, save_path = "playgorund/obs_c_exp_rf_scatter.pdf")

# Expected counts after calibration
In the test set only, compared with observed counts in the test set

In [ ]:
fig, ax = plt.subplots(figsize = (6,4))
plt.style.use("bmh")
# Plots #
    # Plot histogram
X_result["c_exp_car"].plot(kind = "hist", density = False, alpha = 0.8, bins = 70) # change density to true, because KDE uses density

# X #
ax.set_xlabel("Counts")

# Y #
#ax.set_yticks([])
    # Relabel the axis as "Frequency"
ax.set_ylabel("Frequency")

# Overall #
#ax.set_title("Observed traffic counts")
# Remove ticks and spines
ax.tick_params(left = False, bottom = False)
for ax, spine in ax.spines.items():
    spine.set_visible(False)

plt.grid(False)
plt.rcParams['font.family'] = ['serif']
#fig.savefig("figures/c_exp_hist_test158.pdf", bbox_inches = "tight", pad_inches = 0, dpi=10, format = "pdf")
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize = (6,4))
plt.style.use("bmh")
# Plots #
    # Plot histogram
X_result["obs_car"].plot(kind = "hist", density = False, alpha = 0.8, bins = 70) # change density to true, because KDE uses density

# X #
ax.set_xlabel("Counts")

# Y #
#ax.set_yticks([])
    # Relabel the axis as "Frequency"
ax.set_ylabel("Frequency")

# Overall #
#ax.set_title("Observed traffic counts")
# Remove ticks and spines
ax.tick_params(left = False, bottom = False)
for ax, spine in ax.spines.items():
    spine.set_visible(False)

plt.grid(False)
plt.rcParams['font.family'] = ['serif']
#fig.savefig("figures/obs_hist_test158.pdf", bbox_inches = "tight", pad_inches = 0, dpi=10, format = "pdf")
plt.show()

#### applying model to full dataset
This is not really representative, because the additional ~30% are likely due to overfitting the train data, which is also part of the full dataset

In [ ]:
drop_columns = ['const', 'c_cb',
      'popdens_motorway', 'maxlanes_maxlanes1500',
       'popdens_Zuid-Holland', 'popdens_Noord-Holland', 'popdens_Utrecht',
       'maxlanes12000_Zuid-Holland', 'maxlanes12000_Noord-Holland',
       'maxlanes12000_Utrecht', 'maxlanes12000_Noord-Brabant', 
       'exp_car_isq','obs_car', 'uv', 'minlanes12000m_bufferzone', 'total_length12000m_bufferzone', 'maxmaxspeed', 'is_bridge']
X_result = X.drop(columns = drop_columns)
X_result["cuberoot_ratio"] = rf6[0].predict(X_result)
X_result["ratio_hat"] = X_result["cuberoot_ratio"]**3
X_result["c_exp_car"] = X_result["exp_car"] * X_result["ratio_hat"] #weight is expected car
X_result["obs_car"] = X["obs_car"]
y = X_result["obs_car"] ** (1/2) 
x = X_result["c_exp_car"] ** (1/2)

plot_result(x = x, y = y, x_label = "$\sqrt{\hat{c} * exp}$", y_label = "$\sqrt{obs}$", save_fig = False, save_path = None)

# Visualize C* of the test set on the map of the road network
This is the ratio of observed counts to <b> calibrated </b>  expected counts.

In [ ]:
#I am doing this now with X_result as X_test
X_result["cs"] = X_result["obs_car"] / X_result["c_exp_car"]
X_result["uv"] = X_test["uv"]
X_calibrated = X_result[["c_exp_car", "cs", "uv"]]

In [ ]:
edges_calibrated = edges_model.merge(X_calibrated, on = "uv", how = "inner")

In [ ]:
edges_calibrated["cs_cb"] = edges_calibrated["cs"] ** (1/3)
edges_calibrated["cs_cb"].hist(bins = 100) 

#### create colormap

In [ ]:
#colorscale for c and c*
#first normalize variables even more 
edges_calibrated["c_sqrt"] = edges_calibrated.c_cb** (1/2)
edges_calibrated["cs_sqrt"] = edges_calibrated.cs_cb** (1/2)

#there is one case in cs sqrt which is lower than the min of c_sqrt. This causes problems in the binning. I will therefore replace this value in cs_sqrt with the min of c_sqrt. For the quality measure interpretation, the difference is meaningless, but for plotting, it is necessary
edges_calibrated["cs_sqrt"] = np.where(edges_calibrated["cs_sqrt"] < edges_calibrated["c_sqrt"].min(), edges_calibrated["c_sqrt"], edges_calibrated["cs_sqrt"])
#create 100 bins and fill them with c_sqrt
c_binned, bins = pd.cut(edges_calibrated.c_sqrt, 100, retbins = True, labels = False)
cs_binned = pd.cut(edges_calibrated.cs_sqrt, bins = bins, labels = False, include_lowest = True)

#add to DF but add +1 to each value to keep 0 as lowest
edges_calibrated["c_plot"] = c_binned +1
edges_calibrated["cs_plot"] = cs_binned +1

#create colorscales
# Diverging color scale to see both below and above 1 values, set lowest value to grey colour
# colormap for c
seismic = cm.get_cmap('seismic', 101)
seismic_colors = seismic(np.linspace(0.175, 1, 101))
darkgrey = np.array([0.5, 0.5, 0.5, 0.15]) #the fourth value sets the transparency of the color. This is necessary, because the grey roads would otherwise cover up the colored roads
seismic_colors[:1 :] = darkgrey
seismiccmp = ListedColormap(seismic_colors)

#colormap for c*
#get min and max of cs_plot to cut off top and bottom of scale, multiply by 0.01 to get value between 0 and 1
cs_min = edges_calibrated["cs_plot"].min() * 0.01
cs_max = edges_calibrated["cs_plot"].max() * 0.01

seismic_colors2 = seismic(np.linspace(cs_min+ 0.175, cs_max, 101))
seismic_colors2[:1 :] = darkgrey

seismiccmp2 = ListedColormap(seismic_colors2)

plot_examples([seismiccmp, seismiccmp2])

In [ ]:
# no grey for colorbar legend
seismic_colors2ng = seismic(np.linspace(cs_min+ 0.175, cs_max, 101))
seismiccmp_cb2 = ListedColormap(seismic_colors2ng)

In [ ]:
plot_examples([seismiccmp_cb2], lo = edges_calibrated["cs_plot"].min(), hi = edges_calibrated["cs_plot"].max())

In [ ]:
plt.scatter(edges_calibrated["cs_plot"], edges_calibrated["cs"], s = 1) #in this plot, the midpoint is somewhere around 42

In [ ]:
nodes, edges = ox.graph_to_gdfs(Ngraph) #get full list of edges back from Ngraph (edges2 only contains edges with sensors)

In [ ]:
edges_calibrated["node_from"] = edges_calibrated.u
edges_calibrated["node_to"] = edges_calibrated.v

In [ ]:
edges = add_features_to_edge(edges_calibrated, edges, features = ["cs_plot"], indexerror = 0)

In [ ]:
graph = ox.graph_from_gdfs(nodes, edges)
ec = ox.plot.get_edge_colors_by_attr(graph, attr="cs_plot", cmap= seismiccmp2)

fig, ax = ox.plot_graph(graph, node_color="w", node_edgecolor="k", node_size=0, edge_color=ec, edge_linewidth=2, show = False) 

# add colorbar
sm = mpl.cm.ScalarMappable(cmap = seismiccmp_cb2)
cb = fig.colorbar(cm.ScalarMappable(cmap = seismiccmp_cb2), ax = ax, location = 'right', shrink = 0.8, ticks = [0, 42*0.01 + 0.275 , 1])
cb.ax.set_yticklabels([int(edges_calibrated.cs.min()), '1', int(edges_calibrated.cs.max())])
cb.ax.set_ylabel('C*', rotation = 0)


#fig.savefig("playgorund/c_map.pdf", format = "pdf", bbox_inches = "tight", pad_inches = 0, dpi = 1000) #bbox_inches and pad_inches ensure that there is no white frame around the plot
plt.show()

### predict obs counts with exp counts and backtransform to compute average error
I did not put this into the thesis, because I didn't have the time anymore. But it is worthwhile to check, because it gives you the average deviation from the observed value in the unit of cars. 

In [ ]:
X_result = X_test.copy()
X_result["ratio_hat"] = rf6[2]**3
X_result["c_exp_car"] = X_result["exp_car"] * X_result["ratio_hat"] #weight is expected car

In [ ]:
X_result["sq_obs_car"] = X_result["obs_car"] ** (1/2) 
X_result["sq_c_exp_car"] = X_result["c_exp_car"] ** (1/2)

y = X_result["sq_obs_car"]
x = X_result[["const", "sq_c_exp_car"]]
o1 = sm.OLS(y, x).fit()

In [ ]:
y_pred = o1.predict(x)
X_result["backtransformedypred"] = y_pred ** 2

In [ ]:
X_result["error_obsexp"] =  X_result["backtransformedypred"] - X_result["obs_car"]
X_result["error_obsexp2"] = (X_result["backtransformedypred"] - X_result["obs_car"])**2

X_result["error_obsexp2_n"] = (( X_result["backtransformedypred"] - X_result["obs_car"])**2) / len(X_result)
my_rmse = (sum(X_result["error_obsexp2_n"]))**(1/2)
print(my_rmse)

In [ ]:
X_result["error_obsexp"].hist(bins = 100)

In [ ]:
X_result["error_obsexp"].describe()

In [ ]:
err = X_result["error_obsexp2"]**(1/2)
err.describe()

In [ ]:
(sum((( X_result["backtransformedypred"] - X_result["obs_car"])**2) / len(X_result)))**(1/2)

In [ ]:
errorbefore = X_result["exp_car"] - X_result["obs_car"]
errorbefore.hist(bins = 100)

In [ ]:
errorbefore.describe()